In today's puzzle we time travel (sic!) to the year 1518 where we need to avoid the guard. She, the guard, has a very organized way of moving and using those rules, we need to find the best way to avoid her. To read the full puzzle, check out the [AoC 2024 Day 6 puzzle](https://adventofcode.com/2024/day/6).

And here's how Chatty, with my instructions, envisions the 1518 elf world:



## Part 1: Mapping guards 

In [74]:
from pathlib import Path
import numpy as np

from typing import List, Tuple

tiny_example = """
..#.
#.^#
....
"""


def parse_input(input) -> Tuple[np.ndarray, Tuple[int, int], np.ndarray]:
    if Path(input).exists():
        with open(input, "r") as f:
            input = f.read()

    output_grid = []
    guard = ()

    guard_dir = {"^": [-1, 0], ">": [0, 1], "v": [1, 0], "<": [0, -1]}
    guard_dir = {k: np.array(v) for k, v in guard_dir.items()}

    for i, line in enumerate(input.strip().split("\n")):
        # check for guard
        output_grid.append([1 for _ in range(len(line))])
        for j, char in enumerate(line):
            if char not in [".", "#"]:
                if guard:
                    raise ValueError(
                        f"Multiple guards found at {guard} and {i, j}"
                    )
                guard = (i, j)
                guard_direction = guard_dir[char]
            elif char == "#":
                output_grid[i][j] = 0
    return np.array(output_grid), np.array(guard), guard_direction


parse_input(tiny_example)

(array([[1, 1, 0, 1],
        [0, 1, 1, 0],
        [1, 1, 1, 1]]),
 array([1, 2]),
 array([-1,  0]))

In [62]:
def guard_walk(
    grid: np.ndarray,
    guard_position: np.ndarray | list,
    guard_direction: np.ndarray | list,
):
    max_x, max_y = grid.shape
    visited = set()
    rot = np.array([[0, -1], [1, 0]])
    while True:
        visited.add(tuple(guard_position))
        # given the guard direction, check the next step
        next_step = guard_position + guard_direction
        # print(f"guards position: {guard_position}, next step: {next_step}")
        # check if next step is out of bounds
        if (
            next_step[0] < 0
            or next_step[0] >= max_x
            or next_step[1] < 0
            or next_step[1] >= max_y
        ):
            # finished walking
            return visited
        # print(f"next vaalue: {grid[next_step[0], next_step[1]]}")
        # check if next step is a wall
        if grid[next_step[0], next_step[1]] == 0:
            # rotate direction by 90 degrees right
            guard_direction = guard_direction @ rot

        else:
            # move to next step
            guard_position = next_step


len(guard_walk(*parse_input(tiny_example)))

2

In [63]:
len(guard_walk(*parse_input("../inputs/example_day_06.txt")))

41

In [66]:
%time
len(guard_walk(*parse_input("../inputs/day_06.txt")))

CPU times: user 2 μs, sys: 1 μs, total: 3 μs
Wall time: 11 μs


4454

## Part 2: Trap the guard

In this task we need to trap the guard in a infinite loop. 



In [ ]:
# if the guard visits start again with the same direction, that means it's a loop
# i need to keep track of the original direction and starting position

In [135]:
def print_grid(grid):
    d_ = {1: ".", 0: "X"}
    for row in grid:
        print("".join(d_[i] for i in row))

print_grid(grid)

XXX
.X.
X.X


In [138]:
def move(position, direction, visited, grid, obstacle_inserted=False):
    #print(f"position: {position}")
    #print(f"direction: {direction}")
    #print()
    rot = np.array([[0, -1], [1, 0]])
    new_position = position + direction
    #print(f"grid shape: {grid.shape}")
    #print(f"new_position: {new_position}")
    #print(f"and visited: {visited}")

    if any(new_position >= grid.shape) or any(new_position < 0):
        return 0

    elif any(
        [
            np.array_equal(direction, d)
            for d in visited.get(tuple(new_position), [])
        ]
    ):
        print(visited.keys())
        print_grid(grid)
        return 1

    elif grid[new_position[0], new_position[1]] == 0:
        return move(position, direction @ rot, visited, grid, obstacle_inserted)

    elif grid[new_position[0], new_position[1]] == 1 and obstacle_inserted:
        # already inserted an obstacle, so no inserting, just drifting
        return move(new_position, direction, visited, grid, obstacle_inserted)

    else:  # not inserted, so we will split between inserting and not inserting
        # if not inserted
        not_inserted_visited = visited.copy()

        updated_dirs = not_inserted_visited.get(
            (int(new_position[0]), int(new_position[1])), []
        )
        if not any([np.array_equal(direction, d) for d in updated_dirs]):
            updated_dirs.append(direction)

        not_inserted_visited[(int(new_position[0]), int(new_position[1]))] = (
            updated_dirs
        )

        new_grid = grid.copy()
        new_grid[new_position[0], new_position[1]] = 0
        # if inserted, then return the same, but insert obstacle at the grid next_position
        return move(
            new_position,
            direction,
            not_inserted_visited,
            grid,
            obstacle_inserted,
        ) + move(position, direction, visited, new_grid, obstacle_inserted=True)


g, p, d = parse_input("../inputs/example_day_06.txt")

In [116]:
any(np.array([0,1]) < 0)

False

In [139]:
move(p, d, {}, g)

dict_keys([(5, 4), (4, 4), (3, 4), (2, 4), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (6, 7), (6, 6), (6, 5), (6, 4), (6, 3), (6, 2), (5, 2), (4, 2), (4, 3), (4, 5), (4, 6), (5, 6), (7, 6), (8, 6), (8, 5), (8, 4), (8, 3), (8, 2), (8, 1), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 7), (8, 7)])
....X.....
.........X
..........
..X.......
.......X..
..........
.X........
........X.
X.........
......XX..
dict_keys([(5, 4), (4, 4), (3, 4), (2, 4), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (6, 7), (6, 6), (6, 5), (6, 4), (6, 3), (6, 2), (5, 2), (4, 2), (4, 3), (4, 5), (4, 6), (5, 6), (7, 6), (8, 6), (8, 5), (8, 4), (8, 3), (8, 2), (8, 1), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5)])
....X.....
.........X
..........
..X.......
.......X..
..........
.X........
.......XX.
X.........
......X...
dict_keys([(5, 4), (4, 4), (3, 4), (2, 4), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (6, 7), (

9

In [78]:
a = np.array([1, 0])
tuple(a)

(np.int64(1), np.int64(0))

In [101]:
np.array_equal(a, np.array([0, 1]))

False

In [71]:
grid = np.array([[0, 0, 0], [1, 0, 1], [0, 1, 0]])

In [73]:
any(a > grid.shape)

False